# !!! RUN download_ACS.ipynb AND DOWNLOAD CRIME DATASET MANUALLY BEFORE RUNNING THIS WORKBOOK OTHERWISE YOU'LL HAVE NO DATA :)

In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from census import Census
from us import states

In [2]:
raw_data_dir = "data/raw"

raw_acs_path = os.path.join(raw_data_dir, "acs5_2023_raw.csv")
raw_incidents_path = os.path.join(raw_data_dir, "Crime_Data_from_2020_to_Present.csv")

## Crime Incidents Data

In [3]:
# Read raw data
incidents_raw = pd.read_csv(raw_incidents_path)
print(len(incidents_raw), "records in raw crime incidents data")

1004991 records in raw crime incidents data


In [4]:
# Filter out invalid lat/lon values -- online it says locations errors or missings are encoded as 0, 0 for lat/lon
incidents = incidents_raw[~((incidents_raw['LAT'] == 0) & (incidents_raw['LON'] == 0))]
print(len(incidents), "incidents after dropping invalid latitude/longitude values")
print(len(incidents_raw) - len(incidents), "records dropped")

1002751 incidents after dropping invalid latitude/longitude values
2240 records dropped


In [5]:
# Convert to GeoDataFrame
incidents_geo = gpd.GeoDataFrame(
    incidents,
    geometry=gpd.points_from_xy(incidents["LON"], incidents["LAT"]),
    crs="EPSG:4326"
)

## ZCTA Boundary Data

In [6]:
# Load ZCTA data from Census
# -- Using 2023 because this is the most recent census data year and they should match
zctas = gpd.read_file(
    "https://www2.census.gov/geo/tiger/TIGER2023/ZCTA520/tl_2023_us_zcta520.zip"
)
zctas = zctas.to_crs("EPSG:4326")
zctas = zctas.rename(columns={"ZCTA5CE20": "ZCTA"})
print(f"Loaded {len(zctas)} ZIP code areas in the US")

Loaded 33791 ZIP code areas in the US


## Join ZCTA Boundary Data to Crime Incidents based on Latitude/Longitude

In [7]:
# Spatial join to merge ZCTA code if the lat/lon point is within the ZCTA boundary
# -- Not exactly sure how polygons/shape files work tbh, but have done a similar process to load in ACS data at work
incidents_geo = incidents_geo.to_crs(zctas.crs)
incidents_zcta = gpd.sjoin(incidents_geo, zctas, how = "left", predicate = "within")

In [8]:
# Drop crimes that we can't map to a zip code
incidents_zcta = incidents_zcta[~incidents_zcta["ZCTA"].isnull()]
print(len(incidents_zcta), "incidents after dropping those that couldn't be mapped to a zip code")
print(len(incidents) - len(incidents_zcta), "records dropped")
print(len(incidents_zcta["ZCTA"].unique()), "ZCTAs")

n_incidents_geocoded = len(incidents_zcta)

1002345 incidents after dropping those that couldn't be mapped to a zip code
406 records dropped
150 ZCTAs


## Crime Incidents Data Prep

In [9]:
# Create violent/non-violent classification
# -- Taken from Vikram's code

# Make a list of crimes that we identified as Violent from one of the previous steps

violent_crimes = ['CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT', 'INTIMATE PARTNER - SIMPLE ASSAULT', 'ROBBERY', 'CRIMINAL HOMICIDE', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT', 'RAPE, FORCIBLE', 'MANSLAUGHTER, NEGLIGENT', 'BATTERY - SIMPLE ASSAULT', 'LYNCHING']

# Create a new Crime Type column with Violent and Non-Violent labels
incidents_zcta['Crime Type'] = np.where(incidents_zcta['Weapon Used Cd'].notnull() | incidents_zcta['Crm Cd Desc'].isin(violent_crimes), 'Violent', 'Non-Violent')

In [10]:
# Convert incident date to pandas datetime
incidents_zcta["incident_date"] = pd.to_datetime(incidents_zcta['DATE OCC'], format = "%m/%d/%Y %I:%M:%S %p")

# Filter prior to January 2024 -- this is when there was a reporting change and the reporting drops drastically because LAPD switched to a new system
incidents_zcta = incidents_zcta[incidents_zcta["incident_date"] < "2024-01-01"]
print(len(incidents_zcta), "incidents after dropping those after the reporting date change")
print(n_incidents_geocoded - len(incidents_zcta), "records dropped")

# Get month/year from date
incidents_zcta["incident_month"] = incidents_zcta['incident_date'].dt.to_period('M')

# Aggregate incident data to zip code and month
incidents_zcta_month = incidents_zcta.groupby(["ZCTA", "incident_month"])["Crime Type"].value_counts().unstack(fill_value = 0).add_prefix("crime_").reset_index()
print(len(incidents_zcta_month), "records after aggregating to ZCTA-month level")

874725 incidents after dropping those after the reporting date change
127620 records dropped
6265 records after aggregating to ZCTA-month level


In [11]:
# Fill in missing months in the data
# -- There may be certain months that a zip code didn't have any crime, so there isn't a row
# -- We need a row for every zip/month combo, and we can fill crime counts with 0s

# Create date formatted column
incidents_zcta_month["incident_month_timestamp"] = incidents_zcta_month['incident_month'].dt.to_timestamp()

# List of all ZCTAs
incidents_zctas = incidents_zcta_month['ZCTA'].unique()

# List of all months between min and max month in data
all_months = pd.date_range(incidents_zcta_month["incident_month_timestamp"].min(), incidents_zcta_month["incident_month_timestamp"].max(), freq = "MS")

# Get all combinations of ZCTAs and months
all_zctas_months = pd.MultiIndex.from_product([incidents_zctas, all_months], names = ["ZCTA", "incident_month_timestamp"])

# Reindex your dataframe
incidents_zcta_month_fill = incidents_zcta_month.set_index(["ZCTA", "incident_month_timestamp"]).reindex(all_zctas_months).reset_index()

# Clean up df
incidents_zcta_month_fill = incidents_zcta_month_fill.drop(["incident_month"], axis = 1)
incidents_zcta_month_fill = incidents_zcta_month_fill.fillna(0)

incidents_zcta_month_fill

Crime Type,ZCTA,incident_month_timestamp,crime_Non-Violent,crime_Violent
0,90001,2020-01-01,48.0,38.0
1,90001,2020-02-01,34.0,12.0
2,90001,2020-03-01,30.0,16.0
3,90001,2020-04-01,16.0,25.0
4,90001,2020-05-01,31.0,22.0
...,...,...,...,...
7147,91801,2023-08-01,0.0,0.0
7148,91801,2023-09-01,0.0,0.0
7149,91801,2023-10-01,0.0,0.0
7150,91801,2023-11-01,0.0,0.0


In [12]:
# FEATURE CALCULATIONS

# Calculate lagging indicators
incidents_zcta_month_fill["violent_lag_1"] = incidents_zcta_month_fill.groupby("ZCTA")["crime_Violent"].shift(1)
incidents_zcta_month_fill["violent_lag_2"] = incidents_zcta_month_fill.groupby("ZCTA")["crime_Violent"].shift(2)
incidents_zcta_month_fill["violent_lag_3"] = incidents_zcta_month_fill.groupby("ZCTA")["crime_Violent"].shift(3)

# -- Drop records that have nulls for lagging indicators -- we only want to keep records in our dataset that we have a complete history for
incidents_zcta_month_fill = incidents_zcta_month_fill.dropna(subset = ["violent_lag_1", "violent_lag_2", "violent_lag_3"]).copy()

# Calculate season flag
def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

incidents_zcta_month_fill["month"] = incidents_zcta_month_fill["incident_month_timestamp"].dt.month
incidents_zcta_month_fill["season"] = incidents_zcta_month_fill["month"].apply(get_season)

# Create sine/cosine transformations of month so model recognizes cyclical time periods
incidents_zcta_month_fill["month_sin"] = np.sin(2 * np.pi * incidents_zcta_month_fill["month"] / 12)
incidents_zcta_month_fill["month_cos"] = np.cos(2 * np.pi * incidents_zcta_month_fill["month"] / 12)

print(len(incidents_zcta_month_fill), "rows after aggregating to the ZCTA/month level, filling in missing months per ZCTA, and keeping a month range such that every month we're predicting for has a full 3-month prior history to create lagging indicators")
incidents_zcta_month_fill

6705 rows after aggregating to the ZCTA/month level, filling in missing months per ZCTA, and keeping a month range such that every month we're predicting for has a full 3-month prior history to create lagging indicators


Crime Type,ZCTA,incident_month_timestamp,crime_Non-Violent,crime_Violent,violent_lag_1,violent_lag_2,violent_lag_3,month,season,month_sin,month_cos
3,90001,2020-04-01,16.0,25.0,16.0,12.0,38.0,4,spring,8.660254e-01,-5.000000e-01
4,90001,2020-05-01,31.0,22.0,25.0,16.0,12.0,5,spring,5.000000e-01,-8.660254e-01
5,90001,2020-06-01,29.0,36.0,22.0,25.0,16.0,6,summer,1.224647e-16,-1.000000e+00
6,90001,2020-07-01,37.0,27.0,36.0,22.0,25.0,7,summer,-5.000000e-01,-8.660254e-01
7,90001,2020-08-01,24.0,29.0,27.0,36.0,22.0,8,summer,-8.660254e-01,-5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...
7147,91801,2023-08-01,0.0,0.0,0.0,0.0,0.0,8,summer,-8.660254e-01,-5.000000e-01
7148,91801,2023-09-01,0.0,0.0,0.0,0.0,0.0,9,fall,-1.000000e+00,-1.836970e-16
7149,91801,2023-10-01,0.0,0.0,0.0,0.0,0.0,10,fall,-8.660254e-01,5.000000e-01
7150,91801,2023-11-01,0.0,0.0,0.0,0.0,0.0,11,fall,-5.000000e-01,8.660254e-01


In [13]:
print(len(incidents_zcta_month_fill["ZCTA"].unique()), "zip codes in cleaned and aggregated crime incident data")
print("Date range:", incidents_zcta_month_fill["incident_month_timestamp"].min(), "to", incidents_zcta_month_fill["incident_month_timestamp"].max())

149 zip codes in cleaned and aggregated crime incident data
Date range: 2020-04-01 00:00:00 to 2023-12-01 00:00:00


## ACS Data

In [14]:
# Read in ACS data
acs_raw = pd.read_csv(raw_acs_path)
acs_raw.head()

,Unnamed: 0,NAME,pop_total,median_household_income,gini_index,pop_labor_force,pop_unemployed,pop_poverty_status,pop_below_fpl,zip code tabulation area
0,0,ZCTA5 00601,16721,18571,0.4460,6059,1301,16676,10199,601
1,1,ZCTA5 00602,37510,21702,0.4747,12328,756,37419,17504,602
2,2,ZCTA5 00603,48317,19243,0.5599,16358,2729,47655,22683,603
3,3,ZCTA5 00606,5435,20226,0.4552,1414,0,5435,2984,606
4,4,ZCTA5 00610,25413,23732,0.4980,9876,625,25312,11145,610


In [15]:
# ACS data calculations/cleanup

acs = acs_raw.copy()

# Get zip code on it's own (keep as string to match with zcta table above)
acs["ZCTA"] = acs["NAME"].str.split(' ', n = 1).str[1]

# Calculate unemployment rate
acs["unemployment_rate"] = acs["pop_unemployed"] / acs["pop_labor_force"]

# Calculate poverty rate
acs["poverty_rate"] = acs["pop_below_fpl"] / acs["pop_poverty_status"]

# Only keep necessary columns that we want to join
acs_vars_keep = ["ZCTA", "pop_total", "median_household_income", "gini_index", "unemployment_rate", "poverty_rate"]
acs = acs[acs_vars_keep]

# Confirm no duplicate ZCTAs
print(acs.duplicated(subset = ["ZCTA"]).sum(), "ZCTA duplicates")
acs.head()

0 ZCTA duplicates


,ZCTA,pop_total,median_household_income,gini_index,unemployment_rate,poverty_rate
0,00601,16721,18571,0.4460,0.214722,0.611598
1,00602,37510,21702,0.4747,0.061324,0.467784
2,00603,48317,19243,0.5599,0.166830,0.475984
3,00606,5435,20226,0.4552,0.000000,0.549034
4,00610,25413,23732,0.4980,0.063285,0.440305


## Merge ACS Data to Incidents

In [16]:
incidents_final = pd.merge(incidents_zcta_month_fill, acs, on = "ZCTA", how = "left")

print(len(incidents_zcta_month_fill), "rows in cleaned incidents data")
print(len(acs), "rows in ACS data")
print(len(incidents_final), "rows in merged data")

6705 rows in cleaned incidents data
33772 rows in ACS data
6705 rows in merged data


In [17]:
# Replace invalid data values with NULLs so they don't mess up EDA
print("Zip codes with some missing ACS data:", incidents_final[(incidents_final[acs_vars_keep[1:]] < 0).any(axis = 1)]["ZCTA"].unique())
incidents_final[acs_vars_keep[1:]] = incidents_final[acs_vars_keep[1:]].mask(incidents_final[acs_vars_keep[1:]] < 0)
incidents_final.describe()
incidents_final.info()

Zip codes with some missing ACS data: ['90052' '90071' '90073' '90079' '90089' '90095' '91330' '91371' '91608']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6705 entries, 0 to 6704
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ZCTA                      6705 non-null   object        
 1   incident_month_timestamp  6705 non-null   datetime64[ns]
 2   crime_Non-Violent         6705 non-null   float64       
 3   crime_Violent             6705 non-null   float64       
 4   violent_lag_1             6705 non-null   float64       
 5   violent_lag_2             6705 non-null   float64       
 6   violent_lag_3             6705 non-null   float64       
 7   month                     6705 non-null   int32         
 8   season                    6705 non-null   object        
 9   month_sin                 6705 non-null   float64       
 10  month_cos                 6705 n

In [18]:
# Only keep features that we will be using for EDA, modeling, and error analysis
keep = ["ZCTA", "incident_month_timestamp", "crime_Non-Violent", "crime_Violent", "violent_lag_1", "violent_lag_2", "violent_lag_3", "season", "month_sin", "month_cos", "pop_total", "median_household_income", "gini_index", "unemployment_rate", "poverty_rate"]
incidents_final = incidents_final[keep]

In [19]:
# Train/validation/test split (60/20/20)
# -- Do this by dates instead of randomly
timeline = incidents_final["incident_month_timestamp"].sort_values().unique()
n_periods = len(timeline)

# -- Determine train/val/test cutoffs
train_cutoff_idx = int(0.6 * n_periods)
val_cutoff_idx = int(0.8 * n_periods)

train_end_date = timeline[train_cutoff_idx - 1]
val_end_date = timeline[val_cutoff_idx - 1]

incidents_train = incidents_final[incidents_final["incident_month_timestamp"] <= train_end_date]
incidents_val = incidents_final[(incidents_final["incident_month_timestamp"] > train_end_date) & (incidents_final["incident_month_timestamp"] <= val_end_date)]
incidents_test = incidents_final[incidents_final["incident_month_timestamp"] > val_end_date]

In [20]:
print("Date range for training data:", incidents_train["incident_month_timestamp"].min(), "to", incidents_train["incident_month_timestamp"].max())
print("Number of records in training data:", len(incidents_train))
print("\nDate range for validation data:", incidents_val["incident_month_timestamp"].min(), "to", incidents_val["incident_month_timestamp"].max())
print("Number of records in validation data:", len(incidents_val))
print("\nDate range for test data:", incidents_test["incident_month_timestamp"].min(), "to", incidents_test["incident_month_timestamp"].max())
print("Number of records in test data:", len(incidents_test))

Date range for training data: 2020-04-01 00:00:00 to 2022-06-01 00:00:00
Number of records in training data: 4023

Date range for validation data: 2022-07-01 00:00:00 to 2023-03-01 00:00:00
Number of records in validation data: 1341

Date range for test data: 2023-04-01 00:00:00 to 2023-12-01 00:00:00
Number of records in test data: 1341


In [21]:
incidents_final.to_csv("data/processed/data_stack.csv")
incidents_train.to_csv('data/processed/train.csv')
incidents_val.to_csv('data/processed/validation.csv')
incidents_test.to_csv('data/processed/test.csv')

In [22]:
incidents_final

,ZCTA,incident_month_timestamp,crime_Non-Violent,crime_Violent,violent_lag_1,violent_lag_2,violent_lag_3,season,month_sin,month_cos,pop_total,median_household_income,gini_index,unemployment_rate,poverty_rate
0,90001,2020-04-01,16.0,25.0,16.0,12.0,38.0,spring,8.660254e-01,-5.000000e-01,56403,60751.0,0.4160,0.096901,0.205254
1,90001,2020-05-01,31.0,22.0,25.0,16.0,12.0,spring,5.000000e-01,-8.660254e-01,56403,60751.0,0.4160,0.096901,0.205254
2,90001,2020-06-01,29.0,36.0,22.0,25.0,16.0,summer,1.224647e-16,-1.000000e+00,56403,60751.0,0.4160,0.096901,0.205254
3,90001,2020-07-01,37.0,27.0,36.0,22.0,25.0,summer,-5.000000e-01,-8.660254e-01,56403,60751.0,0.4160,0.096901,0.205254
4,90001,2020-08-01,24.0,29.0,27.0,36.0,22.0,summer,-8.660254e-01,-5.000000e-01,56403,60751.0,0.4160,0.096901,0.205254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6700,91801,2023-08-01,0.0,0.0,0.0,0.0,0.0,summer,-8.660254e-01,-5.000000e-01,53730,85549.0,0.4561,0.059655,0.114839
6701,91801,2023-09-01,0.0,0.0,0.0,0.0,0.0,fall,-1.000000e+00,-1.836970e-16,53730,85549.0,0.4561,0.059655,0.114839
6702,91801,2023-10-01,0.0,0.0,0.0,0.0,0.0,fall,-8.660254e-01,5.000000e-01,53730,85549.0,0.4561,0.059655,0.114839
6703,91801,2023-11-01,0.0,0.0,0.0,0.0,0.0,fall,-5.000000e-01,8.660254e-01,53730,85549.0,0.4561,0.059655,0.114839
